<a href="https://colab.research.google.com/github/aswit3/Tensorflow-Tutorial/blob/master/Tensorflow_Tutorial_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://www.tensorflow.org/images/tf_logo_social.png)

# Tensorflow
**TensorFlow is a powerful open source software library for numerical computation, particularly well suited and fine-tuned for large-scale Machine Learning**



In [2]:
!pip3 install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (1.14.0)


# **install TensorFlow**

In [3]:
!pip3 install tensorflow

# For GPU support
**you need to install *tensorflow-gpu* instead of tensorflow**

In [1]:
!pip3 install tensorflow-gpu

     |████████████████████████████████| 377.0MB 54kB/s 


# To test your installation
**type the following command. It should output the version of TensorFlow you
installed.**

In [6]:
!python3 -c 'import tensorflow; print(tensorflow.__version__)'

1.14.0


# ***Creating Your First Graph and Running It in a Session***

![alt text](https://www.oreilly.com/library/view/hands-on-machine-learning/9781491962282/assets/mlst_0901.png)

In [0]:
import tensorflow as tf

x = tf.Variable(3, name = "X")
y = tf.Variable(4, name = "Y")

f = x*x*y + y + 2

1. It is just creates a computational graph. In fact, even the variables are not initialized.

2. To evaluate this graph, you need to open a Tensorflow session and useit to initialize the variables and evaluate f



In [4]:
f

<tf.Tensor 'add_3:0' shape=() dtype=int32>

In [7]:

# creates a session, initializes the variables, and evaluates
# closes the session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [0]:
sess.close()

In [0]:
#Having to repeat sess.run() all the time is a bit cumbersome, 
#but fortunately there is a better way:

with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  result = f.eval()

In [11]:
result

42

1. Calling x.initializer.run() is
equivalent to calling tf.get_default_session().run(x.initializer), and similarly f.eval() is
equivalent to calling tf.get_default_session().run(f). 
2. The session is automatically closed at the end of the block.
3. Instead of manually running the initializer for every single variable, you can use the
*global_variables_initializer()* function.

4. Note that it does not actually perform the initialization
immediately, but rather creates ***a node in the graph that will initialize all variables when it is run***

In [0]:
init = tf.global_variables_initializer() #prepare an init mode

In [0]:
with tf.Session() as sess:
  init.run() #actually initialize all the variables
  #sess.run(init)
  result = f.eval()

In [14]:
result

42

1. **Inside Jupyter or within a Python shell** you may prefer to **create an InteractiveSession**. 
2. The only
difference from a regular Session is that when an InteractiveSession is created it **automatically sets
itself as the default session** , so you don’t need a with block

In [0]:
init = tf.global_variables_initializer()

In [17]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [18]:
f.eval()

42

Tensorflow Program is typically split into two parts

1. the first part builds a computation graph. this is called the **construction phase**. build computational graph.
2. The second part runs it. this is the **execution phase**.. trainig process, model architecture.

# Managing Graphs

Any node you create is automatically added to the default graph

In [0]:
  a = tf.Variable(3)

In [20]:
a

<tf.Variable 'Variable:0' shape=() dtype=int32_ref>

In [21]:
a.graph is tf.get_default_graph()

True

In most cases this is fine, but 
1. sometimes you may want to **manage multiple independent graphs**.
2. You can do
this by creating a new Graph and temporarily making it the default graph inside a with block

In [0]:
graph = tf.Graph()

with graph.as_default():
  a = tf.Variable(3)  

In [28]:
a.graph is graph

True

In [29]:
a.graph is tf.get_default_graph()

False

1. In Jupyter (or in a Python shell), it is common to run the **same commands more than once** while you are experimenting.
2. As a result, you may end up with a default graph **containing many duplicate nodes**. 
3. One solution is to **restart the Jupyter kernel** (or the
Python shell), but 
4. a more convenient solution is to just reset the default graph by running **tf.reset_default_graph()**.

# Lifecycle of a Node Value

in Tensorflow 
1. TensorFlow automatically determines the set of nodes that it depends on and it
evaluates these nodes 
2. A
variable starts its life when its initializer is run, and it ends when the session is closed.


In [30]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

10
15


if you want to evaluate y and z efficiently, without evaluating w and x twice as in the previous code, you
must ask TensorFlow to evaluate both y and z in just one graph run

In [0]:
with tf.Session() as sess:
  y, z = sess.run([y, z])

In [38]:
y, z

(10, 15)

1. In **single-process** TensorFlow, multiple sessions **do not share any state**, even if they reuse the same graph
2. In distributed TensorFlow, **variable state is stored on the servers**, not in
the sessions, so **multiple sessions can share the same variables**.

# Linear Regression
Using the Normal Equation

In [41]:
!pip install scikit-learn

In [48]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf

housing = fetch_california_housing()
m, n = housing.data.shape

print(m, n)
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
print(housing_data_plus_bias.shape)
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)


20640 8
(20640, 9)


In [51]:
from numpy import c_
import numpy
a = numpy.ones(4)
print(a)
b = numpy.zeros((4,10))  
print(b)  
c_[a,b]

[1. 1. 1. 1.]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])